# Berghain Puzzle - Step-by-Step Game Runner

This notebook provides an interactive, step-by-step environment for running the Berghain bouncer puzzle with real-time feedback and visualization.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Import required libraries
import sys
import os

# Add src to path so we can import our modules (go up one level from notebooks/)
sys.path.append('../src')

from berghain import BerghainAPI
from berghain.strategies import (
    RandomStrategy, 
    AlwaysAcceptStrategy, 
    ConstraintAwareStrategy, 
    Scenario1Strategy,
)
from berghain import GameRunner
from berghain import BerghainVisualizer

import pandas as pd
import numpy as np
from dotenv import load_dotenv

# Load environment variables (go up one level to find .env)
load_dotenv('../.env')

print("✅ All imports successful!")


## Setup

Initialize the API client, choose your strategy, and set up the game runner.


In [ ]:
# Initialize API client with the real ListenLabs endpoint
API_BASE_URL = "https://berghain.challenges.listenlabs.ai"

try:
    api = BerghainAPI(base_url=API_BASE_URL)
    print(f"✅ API client initialized with Player ID: {api.player_id}")
except Exception as e:
    print(f"⚠️  API client initialization failed: {e}")
    print("This might happen if the .env file is not found or API is not accessible.")
    api = None

# Initialize visualizer
visualizer = BerghainVisualizer()
print("✅ Visualizer initialized")

# Choose your strategy here
STRATEGY = Scenario1Strategy()  # Change this to test different strategies
print(f"🎯 Selected strategy: {STRATEGY.get_name()}")

# Initialize game runner
if api is not None:
    runner = GameRunner(api, STRATEGY)
    print("✅ Game runner initialized")
else:
    runner = None
    print("⚠️  Game runner not initialized - API not available")


## Start New Game

Choose a scenario and start the game to see constraints and initial setup.


In [ ]:
# Choose scenario and start game
SCENARIO = 1  # Change this to 1, 2, or 3

if runner is not None:
    try:
        game_state = runner.start_game(SCENARIO)
        print(f"\n🎮 Game started for Scenario {SCENARIO}!")
    except Exception as e:
        print(f"❌ Error starting game: {e}")
        game_state = None
else:
    print("⚠️  Cannot start game - runner not available")
    game_state = None


## View Constraints Only

If you want to see just the constraints without the full game info.


In [ ]:
# View just the constraints for this scenario
if runner is not None and runner.game_started:
    runner.display_constraints_only()
else:
    print("⚠️  Start a game first to see constraints")


## Run Game Steps

Execute the game step by step. You can control how many steps to run at once.


In [ ]:
# Run some steps (change the number as needed)
NUM_STEPS = 500  # Change this to run more or fewer steps

if runner is not None and runner.game_started and not runner.game_completed:
    print(f"🚀 Running {NUM_STEPS} steps...")
    step_records = runner.step(NUM_STEPS, show_progress=False)
    print(f"\n✅ Completed {len(step_records)} steps")
    
    # Show updated game status
    runner.display_game_info()
else:
    print("⚠️  Cannot run steps - game not started or already completed")


## Live Visualization

Visualize current game progress and constraint satisfaction in real-time.


In [ ]:
# Create live visualizations
if runner is not None and runner.game_started:
    dashboard = visualizer.create_live_dashboard(runner)
    dashboard.show()
else:
    print("⚠️  No game data to visualize yet")


## Analyze People Data

Examine the people you've seen so far - who was admitted, who was rejected, and their attributes.


In [ ]:
# Analyze people data
if runner is not None and runner.game_started:
    all_people = runner.get_people_seen()
    admitted_people = runner.get_admitted_people()
    rejected_people = runner.get_rejected_people()
    
    print(f"📊 PEOPLE ANALYSIS")
    print(f"Total people seen: {len(all_people)}")
    print(f"Admitted: {len(admitted_people)}")
    print(f"Rejected: {len(rejected_people)}")
    
    if len(all_people) > 0:
        print(f"\n👥 Recent people (last 10):")
        for person in all_people[-10:]:
            decision_emoji = "✅" if person.decision else "❌"
            attrs = ", ".join([f"{k}:{v}" for k, v in person.attributes.items()])
            print(f"  {decision_emoji} Person #{person.person_index}: [{attrs}]")
        
        # Create DataFrame for analysis
        people_df = pd.DataFrame([
            {
                'person_index': p.person_index,
                'decision': p.decision,
                **p.attributes
            }
            for p in all_people
        ])
        
        print(f"\n📈 Admission rates by attribute:")
        for col in people_df.columns:
            if col not in ['person_index', 'decision']:
                true_admits = people_df[(people_df[col] == True) & (people_df['decision'] == True)]
                true_total = people_df[people_df[col] == True]
                
                if len(true_total) > 0:
                    admit_rate = len(true_admits) / len(true_total) * 100
                    print(f"  {col}=True: {admit_rate:.1f}% admitted ({len(true_admits)}/{len(true_total)})")
else:
    print("⚠️  No people data available yet")


## Save Game State

Save your current progress to continue later or for analysis.


In [ ]:
# Save current game state
if runner is not None and runner.game_started:
    try:
        filename = runner.save_game_state()
        print(f"💾 Game state saved to: {filename}")
        
        # Show game summary
        summary = runner.get_game_summary()
        print(f"\n📋 GAME SUMMARY:")
        print(f"Strategy: {summary['strategy']}")
        print(f"Steps taken: {summary['steps_taken']}")
        print(f"Success: {summary['success']}")
        print(f"Game status: {summary['status']}")
        
    except Exception as e:
        print(f"❌ Error saving game state: {e}")
else:
    print("⚠️  No game to save")


## Quick Commands

Useful commands for game management:

- **To run more steps**: Go back to the "Run Game Steps" cell and change `NUM_STEPS`
- **To restart**: Set `runner.reset()` and then start a new game
- **To change strategy**: Modify `STRATEGY` in the setup and create a new runner
- **To view full game info**: Run `runner.display_game_info()`
- **To check remaining people**: Check `runner.game_state.next_person`
